# Chi squared attack on Caesar encryption

In [72]:
import numpy as np
import pandas as pd
import time

In [73]:
def caesar_encryption(alphabet, shift, text, direction):
    # memorize places of spaces
    spaces = [i for i in range(0, len(text)-1) if text[i] == ' ']
    
    # remove spaces from text and uppercase all text
    text = text.replace(" ", "").upper()
    
    # loop over all letters from the text
    returned_text = []
    for letter in list(text):
        for counter, value in enumerate(list(alphabet)):
            
            # encryption : E = (x + shift) mod len(aplhabet)
            if letter==value and direction=='encrypt':
                returned_text.append(alphabet[(counter + shift)%len(list(alphabet))])
                break
                
            # decryption : E = (x - shift) mod len(aplhabet)
            elif letter==value and direction=='decrypt':
                returned_text.append(alphabet[(counter - shift)%len(list(alphabet))])
                break
                
    # Re-insert spaces
    for idx in spaces:
        returned_text.insert(idx,' ')
                
    return "".join(returned_text)

In [74]:
alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
text = 'CE MESSAGE EST CRYPTE AVEC LA METHODE CESAR QUI CONSISTE EN UNE SUBSTITUTION ALPHABETIQUE'

encrypted_txt = caesar_encryption(alpha, 6, text, 'encrypt')
print(encrypted_txt)
decrypted_txt = caesar_encryption(alpha, 6, encrypted_txt, 'decrypt')
print(decrypted_txt)

IK SKYYGMK KYZ IXEVZK GBKI RG SKZNUJK IKYGX WAO IUTYOYZK KT AT YAHYZOZAZOUT GRVNGHKZOWAK
CE MESSAGE EST CRYPTE AVEC LA METHODE CESAR QUI CONSISTE EN UN SUBSTITUTION ALPHABETIQUE


In [75]:
def calculate_chi_score_fr(text, max_n=2):
    # load n_grams and initialize variables
    n_grams = pd.read_csv('n_grams_fr.csv').drop('Unnamed: 0', axis=1).dropna()

    # generate the n_gram list to loop over
    search_gram = np.array(n_grams['n_gram'].loc[n_grams['n'] <= max_n])
    
    # calculate the chi squared score i for each letter
    score_chi2_i = []
    for n_gram in search_gram:
        ci = text.count(n_gram) # Actual count of the letter
        ei = float(n_grams['freq'].loc[n_grams['n_gram']==n_gram])*len(encrypted_txt)
        score_chi2_i.append((ci - ei)**2/ei )
     
    # sum all chi squared score i to calculate total chi squared score 
    return sum(score_chi2_i)

In [76]:
def chi_squared_attack(alphabet, encrypted_txt):
    
    print("Decryption of:",encrypted_txt[:15]+'...\n')
          
    final_score = 0
    decrypted_txt = ''
    hist = pd.DataFrame(columns=['shift', 'chi_2_score','decrypt'])
    
    # loop over each n_gram and test decryption
    for i in range(1,len(alphabet)):
        
        # start chrono
        start = time.time()
        
        # test the i_th shift for decryption
        test_decrypt = caesar_encryption(alpha, i, encrypted_txt, 'decrypt')
        
        # calculate the chi squared score i for each letter and sum
        score_chi2 = calculate_chi_score_fr(test_decrypt)
        
        # end chrono
        end = time.time()
        
        # save history
        hist.loc[i] = [i,score_chi2,test_decrypt]
        
        # keep only best score (inverse score to initialize to 0 and keep highest)
        if 1/score_chi2 > final_score:
            final_score = 1/score_chi2
            decrypted_txt = test_decrypt
    
        # display progress
        print(
            str(i)+'/26:',
            str(round(end - start,2))+'s', # display elapsed time
            'score:',round(score_chi2,3),
            test_decrypt[:15]+'...') # display only 15 characters
    
    # display the best solutions
    print('\n',hist.sort_values('chi_2_score').head())
     
    return decrypted_txt, final_score

In [77]:
chi_squared_attack(alpha, encrypted_txt)

Decryption of: IK SKYYGMK KYZ ...

1/26: 0.35s score: 544739.392 HJ RJXXFLJ JXY ...
2/26: 0.32s score: 375315.322 GI QIWWEKI IWX ...
3/26: 0.35s score: 245880.85 FH PHVVDJH HVW ...
4/26: 0.39s score: 126623.913 EG OGUUCIG GUV ...
5/26: 0.35s score: 29530.933 DF NFTTBHF FTU ...
6/26: 0.3s score: 384.967 CE MESSAGE EST ...
7/26: 0.32s score: 99004.212 BD LDRRZFD DRS ...
8/26: 0.32s score: 296663.346 AC KCQQYEC CQR ...
9/26: 0.3s score: 146139.902 ZB JBPPXDB BPQ ...
10/26: 0.32s score: 150477.905 YA IAOOWCA AOP ...
11/26: 0.3s score: 210879.649 XZ HZNNVBZ ZNO ...
12/26: 0.32s score: 22404.072 WY GYMMUAY YMN ...
13/26: 0.32s score: 201468.799 VX FXLLTZX XLM ...
14/26: 0.3s score: 91221.788 UW EWKKSYW WKL ...
15/26: 0.35s score: 187821.973 TV DVJJRXV VJK ...
16/26: 0.35s score: 177382.497 SU CUIIQWU UIJ ...
17/26: 0.32s score: 124879.772 RT BTHHPVT THI ...
18/26: 0.32s score: 39651.527 QS ASGGOUS SGH ...
19/26: 0.34s score: 49455.75 PR ZRFFNTR RFG ...
20/26: 0.43s score: 138912.015 OQ YQEEM

('CE MESSAGE EST CRYPTE AVEC LA METHODE CESAR QUI CONSISTE EN UN SUBSTITUTION ALPHABETIQUE',
 0.0025976236551025532)